In [1]:
!pip install pandas xlsxwriter

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not install packages due to an OSError: [Errno 30] Read-only file system: '/root/.local/lib'



In [2]:
pip install --upgrade pip

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/runpy.py", line 188, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/local/lib/python3.9/runpy.py", line 147, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/local/lib/python3.9/runpy.py", line 158, in _get_module_details
    code = loader.get_code(mod_name)
  File "<frozen importlib._bootstrap_external>", line 982, in get_code
  File "<frozen importlib._bootstrap_external>", line 1040, in get_data
OSError: [Errno 5] Input/output error
Note: you may need to restart the kernel to use updated packages.


ConnectionRefusedError: [Errno 111] Connection refused

In [4]:
# spark.sql("DROP TABLE IF EXISTS demo.nyc.taxis_10M_50COLUMNS")


In [5]:
# import os
# iceberg_table_dir = "../warehouse/nyc/taxis_10M_50COLUMNS"
# metadata_dir = f"{iceberg_table_dir}/metadata"
# data_dir = f"{iceberg_table_dir}/data"
# input_data_dir = f"../input_data"
# analysis_info = []
# records_before_op = 0

# def append_to_file(file_path, msg):
#     open_mode = "a"
#     if not os.path.exists(file_path):
#         open_mode = "w"

#     # Open the CSV file in write mode
#     with open(file_path, open_mode) as file:
#         writer = csv.writer(file)
        
#         if open_mode=="w":
#             #writing header of the columns
#             writer.writerows([list(msg.keys())])    

#         row_values = [list(msg.values())]
#         # Write the data to the CSV file
#         writer.writerows(row_values)

# def get_size():
#     # List the metadata files
#     manifest_pattern = re.compile(r".*-m\d+\.avro$")
#     metadata_files = os.listdir(metadata_dir)
    
#     # Initialize variables to store the sizes of different types of metadata files
#     snap_avro_size = 0
#     metadata_json_size = 0
#     m_avro_size = 0

#     data_dir_size = 0
#     # get data dir size
#     data_dir_files = os.listdir(data_dir)
#     # print(data_dir_files)
#     for filename in data_dir_files:
#         file_path = os.path.join(data_dir, filename)
#         data_dir_size += os.path.getsize(file_path) / 1024  # Convert size to KB
    
#     # Iterate through the metadata files and calculate their sizes
#     for file in metadata_files:
#         file_path = os.path.join(metadata_dir, file)
#         file_size_kb = os.path.getsize(file_path) / 1024  # Convert size to KB
        
#         if file.startswith("snap-") and file.endswith(".avro"):
#             snap_avro_size += file_size_kb
#         elif file.endswith(".metadata.json"):
#             metadata_json_size += file_size_kb
#         elif manifest_pattern.match(file):
#             m_avro_size += file_size_kb
    
#     # Print the time taken and the sizes of the metadata files
#     # print(f"Time taken to read Parquet files: {time_taken:.2f} seconds")
#     # print(f"Size of snap-*.avro files: {snap_avro_size:.2f} KB")
#     # print(f"Size of *.metadata.json files: {metadata_json_size:.2f} KB")
#     # print(f"Size of *m{0-9}{1,}.avro files: {m_avro_size:.2f} KB")

#     return {"data_dir_size": data_dir_size,"metadata_size": metadata_json_size,"snapshot_size": snap_avro_size,"manifest_size": m_avro_size}


In [6]:
# from pyspark import SparkContext

# # Stop the existing SparkContext if one is active
# sc.stop()

# # Now create a new SparkContext with the updated configuration
# conf = SparkConf().set("spark.local.dir", r"C:\Users\DataEngg\Downloads\Nexus-Data-Iceberg-main\Nexus-Data-Iceberg-main\spark_temp")
# sc = SparkContext(conf=conf)


In [7]:
# from pyspark.sql.types import (
#     DoubleType, FloatType, LongType, StructType, StructField, 
#     StringType, IntegerType, DateType
# )

# # Define the schema with 50 columns based on the required data types
# schema = StructType([
#     # StructField("vendor_id", LongType(), True),  # INT
#     # StructField("trip_id", LongType(), True),  # INT
#     # StructField("trip_distance", FloatType(), True),  # FLOAT
#     # StructField("fare_amount", DoubleType(), True),  # DOUBLE
#     # StructField("store_and_fwd_flag", StringType(), True)  # STRING
# # ] + [
#     # Assigning VARCHAR, INT, STRING, and DATE data types in a cyclic pattern
#     StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 0 else  # VARCHAR
#     StructField(f"extra_col_{i}", IntegerType(), True) if i % 4 == 1 else  # INT
#     StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 2 else  # STRING
#     StructField(f"extra_col_{i}", DateType(), True)  # DATE
#     for i in range(50)
# ])

# # Create an empty DataFrame with the schema
# df = spark.createDataFrame([], schema)

# # Create the Iceberg table
# df.writeTo("demo.nyc.taxis_10M_50COLUMNS").create()


In [10]:
# df = spark.table("demo.nyc.taxis_10M_50COLUMNS")
# df.show()

In [1]:
df = spark.sql("SELECT count(*) FROM demo.nyc.taxis_10M_50COLUMNS")
df.show()

+--------+
|count(1)|
+--------+
|10000000|
+--------+



In [3]:
import time
from pyspark.sql.functions import col, when, rand, expr
from pyspark.sql import SparkSession

# Initialize Spark session with memory optimizations
spark = SparkSession.builder \
    .appName("Optimized Iceberg Update") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

# Load the Iceberg table
df = spark.table("demo.nyc.taxis_10M_50COLUMNS")

# Debugging: Print total row count
total_rows = df.count()
print(f"Total rows in table: {total_rows}")

# Get update percentage from user
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
num_rows = max(1, int(total_rows * update_percentage))  # Ensure at least 1 row is updated

# Define batch size
batch_size = 500  # Reduced batch size to optimize memory usage
num_batches = max(1, (num_rows // batch_size) + (1 if num_rows % batch_size else 0))  # Ensure at least 1 batch

print(f"Updating {num_rows} rows (~{update_percentage*100}%) in {num_batches} batches...")

st = time.time()

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1}/{num_batches}...")

    # Select a batch of unique row IDs without collecting to the driver
    sampled_df = df.select("extra_col_0").distinct().orderBy(rand()).limit(min(batch_size, num_rows))

    # Merge updates instead of overwriting partitions
    updated_batch = df.alias("source").join(sampled_df.alias("updates"), "extra_col_0", "inner") \
        .withColumn("extra_col_1", col("extra_col_1") + 10)

    # Optimize writing using append mode or merge
    updated_batch.repartition(100).writeTo("demo.nyc.taxis_10M_50COLUMNS").append()

    print(f"Batch {batch_num + 1}/{num_batches} updated.")

end = time.time() - st
print(f"\nTotal update time for {num_batches} batches: {end:.2f} sec")


ConnectionRefusedError: [Errno 111] Connection refused

In [9]:
import time, csv
from pyspark.sql.functions import col, when, lit
from pyspark.sql import functions as F
import os

input_data_dir = "../input_data"
output_dir = "../output"
analysis_info = []
records_before_op = 0
total_insertion_time = 0

file_type = input("Enter input file type csv or parquet? : ").lower().strip()
input_data_dir = os.path.join(input_data_dir, file_type)
input_files = os.listdir(input_data_dir)

analysis_file = os.path.join(output_dir, f"analysis_info_{file_type}.csv")
if os.path.exists(analysis_file):
    os.remove(analysis_file)

df = spark.table("demo.nyc.taxis_10M_50COLUMNS")
records_before_op = df.count()

for file in input_files:
    print(f"Started with file={file}")
    file_path = os.path.join(input_data_dir, file)

    st = time.time()
    if file_type == "parquet":
        df = spark.read.parquet(file_path)
    else:
        df = spark.read.csv(file_path, header=True)
        df = df.select(
            F.col("extra_col_0").cast("long"),
            F.col("extra_col_1").cast("int"),
            F.col("extra_col_2").cast("string"),
            F.col("extra_col_3").cast("date"),
            *[F.col(f"extra_col_{i}").cast("string" if i % 4 == 0 or i % 4 == 2 else "int" if i % 4 == 1 else "date") for i in range(4, 45)]
        )

    rows = df.count()
    
    df.writeTo("demo.nyc.taxis_10M_50COLUMNS").append()
    end = time.time() - st
    total_insertion_time += end

    details = {"time_taken": f"{end:.2f} sec", "Operation": f"Inserted {rows} records", "records_after_op": records_before_op + rows}
    records_before_op += rows

    print(f"Inserted {rows} records in {end:.2f} sec.")

# **PRINT INSERTION TIME BEFORE UPDATE**
print(f"\nTotal insertion time: {total_insertion_time:.2f} sec\n")

# === Get Update Percentage from User ===
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100

df = spark.table("demo.nyc.taxis_10M_50COLUMNS")
total_rows = df.count()

num_rows = int(total_rows * update_percentage)
print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# Sample random rows ensuring unique IDs
sampled_df = df.select("extra_col_0").distinct().orderBy(F.rand()).limit(num_rows)
sampled_ids = [row["extra_col_0"] for row in sampled_df.collect()]

st = time.time()

# Updating `extra_col_1` (integer)
updated_df = df.withColumn(
    "extra_col_1",
    when(col("extra_col_0").isin(sampled_ids), col("extra_col_1") + 10)
    .otherwise(col("extra_col_1"))
)

# Updating `extra_col_3` (date) - Commented out
# updated_df = updated_df.withColumn(
#     "extra_col_3",
#     when(col("extra_col_0").isin(sampled_ids), F.date_add(col("extra_col_3"), 5))
#     .otherwise(col("extra_col_3"))
# )

# Overwrite updated data
updated_df.writeTo("demo.nyc.taxis_10M_50COLUMNS").overwritePartitions()

end = time.time() - st

print(f"Updated {num_rows} rows in {end:.2f} sec")


Enter input file type csv or parquet? :  k


FileNotFoundError: [Errno 2] No such file or directory: '../input_data/k'

In [77]:
spark.sql("SELECT * FROM demo.nyc.taxis_1M_50COLUMNS.history")


DataFrame[made_current_at: timestamp, snapshot_id: bigint, parent_id: bigint, is_current_ancestor: boolean]

In [84]:
df = spark.sql("SELECT snapshot_id, made_current_at FROM demo.nyc.taxis_1M_50COLUMNS.history ORDER BY made_current_at DESC")

df.show(

+-------------------+--------------------+
|        snapshot_id|     made_current_at|
+-------------------+--------------------+
|2861666654166238235|2025-02-25 13:10:...|
|7867670594422745254|2025-02-25 13:09:...|
+-------------------+--------------------+



In [ ]:
WITH current_snapshot AS (
    SELECT * FROM demo.nyc.taxis_1M_50COLUMNS.snapshot(2861666654166238235)
),
previous_snapshot AS (
    SELECT * FROM demo.nyc.taxis_1M_50COLUMNS.snapshot(7867670594422745254)
)
SELECT p.*, c.*
FROM previous_snapshot p
FULL OUTER JOIN current_snapshot c
ON p.primary_key_column = c.primary_key_column
WHERE p.primary_key_column IS NULL OR c.primary_key_column IS NULL;


In [85]:
spark.sql("DESCRIBE demo.nyc.taxis_1M_50COLUMNS").show()


+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
| extra_col_0|   string|   NULL|
| extra_col_1|      int|   NULL|
| extra_col_2|   string|   NULL|
| extra_col_3|     date|   NULL|
| extra_col_4|   string|   NULL|
| extra_col_5|      int|   NULL|
| extra_col_6|   string|   NULL|
| extra_col_7|     date|   NULL|
| extra_col_8|   string|   NULL|
| extra_col_9|      int|   NULL|
|extra_col_10|   string|   NULL|
|extra_col_11|     date|   NULL|
|extra_col_12|   string|   NULL|
|extra_col_13|      int|   NULL|
|extra_col_14|   string|   NULL|
|extra_col_15|     date|   NULL|
|extra_col_16|   string|   NULL|
|extra_col_17|      int|   NULL|
|extra_col_18|   string|   NULL|
|extra_col_19|     date|   NULL|
+------------+---------+-------+
only showing top 20 rows

